基础

In [20]:
import tensorflow as tf

In [3]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.84599864, 0.7814544 , 0.8647596 ],
       [0.9622799 , 1.0273397 , 0.53010744],
       [0.8128906 , 0.6991627 , 0.7156985 ]], dtype=float32)>

In [4]:
simple_nn_layer

In [5]:
help(tf.nn.relu)

Help on function relu in module tensorflow.python.ops.gen_nn_ops:

relu(features, name=None)
    Computes rectified linear: `max(features, 0)`.
    
    Args:
      features: A `Tensor`. Must be one of the following types: `float32`, `float64`, `int32`, `uint8`, `int16`, `int8`, `int64`, `bfloat16`, `uint16`, `half`, `uint32`, `uint64`, `qint8`.
      name: A name for the operation (optional).
    
    Returns:
      A `Tensor`. Has the same type as `features`.



In [6]:
def linera_layer(x):
    return 2 * x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linera_layer(x))

deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: id=35, shape=(3,), dtype=int32, numpy=array([3, 5, 7])>

In [8]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

print("square_if_positive(2) = {}".format(square_if_positive(tf.constant(2))))
print("square_if_positive(2) = {}".format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(2) = 0


In [10]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue 
        s += c
    return s

sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=164, shape=(), dtype=int32, numpy=42>

In [11]:
print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sum_even', 'sum_even_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as sum_even_scope:
    s = 0

    def get_state_2():
      return ()

    def set_state_2(_):
      pass

    def loop_body(iterates, s):
      c = iterates
      continue_ = False

      def get_state():
        return ()

      def set_state(_):
        pass

      def if_true():
        continue_ = True
        return continue_

      def if_false():
        return continue_
      cond = c % 2 > 0
      continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('continue_',), ())

      def get_state_1():
        return ()

      def set_state_1(_):
        pass

      def if_true_1():
        s_1, = s,
        s_1 += c
        return s_1

      def if_false_1():
        return s
      cond_1 = ag__.not_(continue_)
 

In [16]:
class CustomModel(tf.keras.models.Model):
    
    @tf.function
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data
        else:
            return input_data // 2

model = CustomModel()
model.call(tf.constant([-2, -4]))

<tf.Tensor: id=231, shape=(2,), dtype=int32, numpy=array([-1, -2])>

In [19]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v+1)
    if v % 3 == 0:
        v.assign(v+1)
    
find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

advaced example: An in_graph training loop

In [27]:
# download data

def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

train_dataset = mnist_dataset()

In [34]:
# definde model

model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()